In [31]:
'''Importing all packages'''
import matplotlib.pyplot as plt
import numpy.random as random
import matplotlib.animation as animation
from scipy import sparse as sparse
from scipy import linalg as ln
import numpy as np
import matplotlib
import os
#matplotlib.use('Qt4Agg')
%matplotlib qt
 
    ####
V_type = 'QHO' # # V_type = ['Free', 'QHO', 'Morse', '...'], is type of potential
epsilon = 50 # disorder
    ####
    
# where to save data to, make sure path has r in front of "" for raw string. Can be False boolean if only animation is required.
save_path = False
# desired name of data file, make sure it include \\ in front of name in "". Must be False if above is False.
file_name = False


'''Creating Gaussian wave packet and controlling its time evolution'''

class Wave_Packet:
    def __init__(self, n_points, dt, V_type, epsilon, to_file, sigma0=1.5, k0=2.5, x0=-0.0, x_begin=-30.0,
                 x_end=30.0):
        # instantiation of constants
        self.to_file = to_file
        self.n_points = n_points
        self.sigma0 = sigma0
        self.k0 = k0
        self.x0 = x0
        self.dt = dt
        self.prob = np.zeros(n_points)
        self.V_type = V_type

        # setting point along x axis, dx = (x_end - x_begin)/n_points
        self.x, self.dx = np.linspace(x_begin, x_end, n_points, retstep=True)

        # equation for a Gaussian wavepacket i.e. Gaussian distribution
        norm = (2.0 * np.pi * sigma0**2)**(-0.25)
        self.psi = np.exp(-(self.x - x0)**2 / (4.0 * sigma0**2))
        self.psi = self.psi*np.exp(1.0j * k0 * self.x)
        self.psi *= (2.0 * np.pi * sigma0**2)**(-0.25)

        self.psi_dis = self.psi

        # setting up potential and disorder
        self.epsilon = epsilon
        N = n_points
        V = np.zeros(N)
        if self.V_type == 'Free':
            pass
        elif self.V_type == 'QHO':
            for i in range(50, 250):
                V[i] = (i-150)**2 / 10000
        elif self.V_type == 'Morse': 
            ebd = 150
            a = 2
            D_e = 1
            for i in range(10, 290):
                V[i] = D_e*(1-np.exp(-a*((i - ebd)/100)))**2
        else:
            pass
        self.potential = V
        epsilonrand =  epsilon*np.random.uniform(-1, 1, size=(N))
        disorder = np.zeros(N)
        disorder[::55] = epsilonrand[::55]
        print(disorder)
            

        # arrays for diagonal and off-diagonal values in Hamiltonian
        diag = np.zeros(N)
        offdiag = np.zeros(N)

        # normal
        # creating  the Hamiltionian
        for i in range(N):
            diag[i] = 1 + V[i]
        for i in range(N):
            offdiag[i] = - 0.5
        H = np.zeros(shape=(N, N))
        for j in range(N):
            per = (j+1) % N  # periodic
            H[j, j] = diag[j]
            H[j, per] = offdiag[j]
            H[per, j] = offdiag[j]
        hamiltonian = H

        # disordered
        # creating  the Hamiltionian with disorder and periodic boundary conditions
        for i in range(N):
            diag[i] = 1 + V[i] + disorder[i]
        for i in range(N):
            offdiag[i] = - 0.5
        H = np.zeros(shape=(N, N))
        for j in range(N):
            per = (j+1) % N  # periodic
            H[j, j] = diag[j]
            H[j, per] = offdiag[j]
            H[per, j] = offdiag[j]
        hamiltonian_dis = H

        I = np.identity(N)

        # Crank-Nicolson method to calculate time evolution of normal wavepacket
        backward = (I - dt / 2.0j * hamiltonian)
        forward = (I + dt / 2.0j * hamiltonian)
        self.evolution_matrix = ln.inv(backward).dot(forward)

        # Crank-Nicolson method to calculate time evolution of disordered wavepacket
        backward_dis = (I - dt / 2.0j * hamiltonian_dis)
        forward_dis = (I + dt / 2.0j * hamiltonian_dis)
        self.evolution_matrix_dis = ln.inv(backward_dis).dot(forward_dis)
        
    def evolve(self):
        # normal
        self.psi = self.evolution_matrix.dot(self.psi)  # steps time evolution
        # normal inner-product
        # abs(self.psi)**2
        self.prob = np.real(self.psi * np.conjugate(self.psi))

        # disordered
        self.psi_dis = self.evolution_matrix_dis.dot(self.psi_dis)
        # disordered inner-product
        self.prob_dis = np.real(self.psi_dis * np.conjugate(self.psi_dis))

        # normal-disordered inner-product
        self.inner = abs(self.psi * np.conjugate(self.psi_dis))  # ?????

        norm = sum(self.prob)  # normalises to begin with, then should be 1
        self.prob /= norm
        self.psi /= norm**0.5

        norm_dis = sum(self.prob_dis)
        self.prob_dis /= norm_dis
        self.psi_dis /= norm_dis**0.5

        self.norm_inner = sum(self.inner)  # ?????
       
        if not self.to_file == False:
        # tries to create new file to write data to
            try:
                with open(self.to_file, 'x') as self.inner_data:
                    self.inner_data.write(str(self.norm_inner) + ' ')

            # if file of that name exists, adds data to that file
            except FileExistsError:
                with open(self.to_file, 'a') as self.inner_data:
                    self.inner_data.write(str(self.norm_inner) + ' ')

                # if file name is None type, carries only the animation without writing to file
            except FileNotFoundError:
                self.norm_inner = self.norm_inner

        return self.prob, self.prob_dis, self.norm_inner 
    
    def sys_data(self):
        if not self.to_file == False:
            try:
                with open(save_path + '\\sys_data_{}_W_{}.txt'.format(self.V_type, self.epsilon), 'x') as sys_data:
                    sys_data.write(str(self.epsilon) + ' ' + str(self.dt)) 
            except FileExistsError:
                with open(save_path + '\\sys_data_{}_W_{}.txt'.format(self.V_type, self.epsilon), 'w') as sys_data:
                    sys_data.write(str(self.epsilon) + ' ' + str(self.dt))      
            return
        else:
            return


'''Creating the animations'''
class Animator:
    def __init__(self, wave_packet, is_finite):
        self.time = 0.0  # infinite runtime time counter
        self.steps = 0  # finite runtime step counter
        self.is_finite = is_finite
        # wave_packet = Wave_Packet(n_points = x, dt = y, epsilon = z)
        self.wave_packet = wave_packet
        self.fig, self.ax = plt.subplots()
        plt.plot(self.wave_packet.x/10, self.wave_packet.potential/10,
                 color='r', alpha=0.2, label='Potential ($E_h/10$)')
        plt.fill_between(self.wave_packet.x/10,
                         self.wave_packet.potential/10, color='r', alpha=0.2)

        self.time_text = self.ax.text(0.05, 0.95, '', horizontalalignment='left',
                                      verticalalignment='top', transform=self.ax.transAxes)
        self.list = []

        # plots x against |Ψ(x)|^2
        self.line, = self.ax.plot(
            self.wave_packet.x/10, self.wave_packet.evolve()[0], label='|Ψ$_0$(x)|$^2$')
        self.line1, = self.ax.plot(self.wave_packet.x/10, self.wave_packet.evolve()[1],
                                   label='|Ψ$_d$(x)|$^2$ (W = {})'.format(self.wave_packet.epsilon))
        self.ax.set_title('Gaussian wavepacket of $\sigma = {}$, $x_0 = {}$ and $k = {}$\n with disorder W = {}'
                          .format(self.wave_packet.sigma0, self.wave_packet.x0, self.wave_packet.k0, self.wave_packet.epsilon))
        self.ax.legend()

        self.ax.set_ylim(0, 0.2)
        self.ax.set_xlabel('Position (a$_0$)')
        self.ax.set_ylabel('Probability density, $|Ψ(x)|^2$')

    def update(self, data1):
        self.line.set_ydata(wave_packet.evolve()[0])
        self.line1.set_ydata(wave_packet.evolve()[1])
        return self.line, self.line1

    def time_step(self):
        if not self.is_finite[0]:
            while True:
                self.time += self.wave_packet.dt
                self.time_text.set_text(
                    'Elapsed time: {:6.2f} fs'.format(self.time * 2.419e-2))  # in atomic (Hartree) units, time is 2.4188...e-17 s

                yield self.wave_packet.evolve()

        if self.is_finite[0]:
            if self.steps <= self.is_finite[1]:
                self.time += self.wave_packet.dt
                self.time_text.set_text(
                    'Elapsed time: {:6.2f} fs'.format(self.time * 2.419e-2))  # in atomic (Hartree) units, time is 2.4188...e-17 s
                self.steps += 1
                yield self.wave_packet.evolve()

    def animate(self):
        self.ani = animation.FuncAnimation(
            self.fig, self.update, self.time_step, interval=1, blit=False)      

        
wave_packet = Wave_Packet(n_points=300, dt=0.25, V_type = V_type,
                          epsilon = epsilon, to_file=save_path + file_name)
# is_finite is [False, 0] for arbitrary animation length
animator = Animator(wave_packet, is_finite=[True, 500])
Wave_Packet(n_points=300, dt=0.25, V_type = V_type, epsilon = epsilon, to_file = save_path + file_name).sys_data()
animator.animate()

plt.show()

[ 39.775837     0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   6.96016537   0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.           0.           0.           0.           0.
   0.         